In [5]:
import pandas as pd
import json
import random

def generate_protocol(input_csv, protocol_name, protocol_type, protocol_group, random_class_order, disconnect_device_on_finish):
    # Read CSV into a DataFrame
    df = pd.read_csv(input_csv).fillna("")

    # Initialize common JSON structure with added protocol_group, random_class_order, and disconnect_device_on_finish parameters
    json_data = {
        "name": protocol_name,
        "parameters": [],
        "active": True,
        "classes": [],
        "randomClassOrder": random_class_order,  # Set the random class order from the function parameter
        "preamble": "",
        "epilogue": "",
        "group": protocol_group,  # Set the group from the function parameter
        "disconnectDeviceOnFinish": disconnect_device_on_finish  # Set the disconnect option from the function parameter
    }

    # Iterate through rows in the CSV
    for index, row in df.iterrows():
        # Extract relevant information from the CSV row
        class_name = row['class_name']
        preamble = row['preamble']
        epilogue = row['epilogue']

        # Check if class_name is 'protocol', then update preamble and epilogue in json_data
        if class_name.lower() == 'protocol':
            json_data['preamble'] = preamble
            json_data['epilogue'] = epilogue
            continue  # Skip further processing for this row

        if protocol_type == 0:
            # Processing for Activity based protocol
            inactive_duration = row['inactive_duration']
            active_duration = row['active_duration']
            activeClass = row['activeClass_name']
            repetitions = int(row['repetitions'])

            class_structure = {
                "class": class_name,
                "intervals": [
                    {"label": "inactive", "durationInMillis": inactive_duration},
                    {"label": activeClass, "durationInMillis": active_duration},
                    {"label": "inactive", "durationInMillis": inactive_duration}
                ],
                "repetitions": repetitions,
                "preamble": preamble,
                "epilogue": epilogue
            }
        elif protocol_type == 1:
            total_duration = int(row['total_duration'])
            stimulus_duration = int(row['stimulus_duration'])
            max_isi = int(row['max_isi'])
            min_isi = int(row['min_isi'])
            r_led = int(row['r_led'])
            g_led = int(row['g_led'])
            b_led = int(row['b_led'])
            led_color = [r_led, g_led, b_led]
            intensity = int(row['intensity'])

            # Generate random isi 
            num_intervals = int(total_duration / (stimulus_duration + (max_isi + min_isi) / 2))
            intervals = []
            for _ in range(num_intervals):
                isi = random.randint(min_isi, max_isi)
                intervals.append(isi)

            # Create the class structure for cognitive test protocol
            class_structure = {
                "class": class_name,
                "intervals": [],
                "preamble": preamble,
                "epilogue": epilogue,
                "type": "REACTION_TIME"
            }

            class_data = class_structure

            for i, duration in enumerate(intervals):
                # Unlabeled interval
                class_data["intervals"].append({
                    "durationInMillis": duration,
                    "ledOptions": {
                        "enabled": False
                    }
                })

                # STIM interval
                class_data["intervals"].append({
                    "label": "STIM",
                    "durationInMillis": stimulus_duration,
                    "ledOptions": {
                        "color": {
                            "red": led_color[0],
                            "green": led_color[1],
                            "blue": led_color[2]
                        },
                        "intensity": intensity,
                        "enabled": True
                    }
                })

            # Set the last interval as an unlabeled interval if the total count is odd
            if len(intervals) % 2 == 1:
                class_data["intervals"].pop()

        # Append the class structure to the classes list
        json_data["classes"].append(class_structure)
        pass

        # Append the class structure to the classes list
        json_data["classes"].append(class_structure)

    # Write the JSON data to a file
    filename = protocol_name + '.json'
    with open(filename, 'w') as json_file:
        json.dump(json_data, json_file, indent=2)
    print ('Please set protocol parameters before uploading your protocol')

# The function now includes additional parameters 'random_class_order' and 'disconnect_device_on_finish' to set these options.



In [4]:
#generate gesture protocol example
randomClassOrder = False
disconnectDeviceOnFinish = False
example_csv = 'activity_example.csv'
generate_protocol(input_csv=example_csv, 
                  protocol_name="test_protocol", 
                  protocol_type=0, 
                  protocol_group="test",
                  random_class_order = False, 
                  disconnect_device_on_finish = True
                  )

NameError: name 'pd' is not defined

In [ ]:
#generate Reaction Time protocol example
example_csv = 'cognitive_test_example.csv' 
generate_protocol(input_csv=example_csv, 
                  protocol_name="cognitive_test_protocol", 
                  protocol_type=1, 
                  protocol_group="test",
                  random_class_order = False, 
                  disconnect_device_on_finish = True)